In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 13.8 MB/s eta 0:00:00


In [6]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
import numpy as np

cols_with_missing = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

df[cols_with_missing] = df[cols_with_missing].replace(0, np.nan)

df.fillna(df.mean(),inplace=True)

print(df.isnull().sum())

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [9]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')

Training set shape: (537, 8)
Test set shape: (231, 8)


In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize

In [11]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2025-08-17 17:04:55,981] A new study created in memory with name: no-name-1e92d16b-0565-43b9-9ec0-f767fbd086c5
[I 2025-08-17 17:04:57,285] Trial 0 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 127, 'max_depth': 14}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-08-17 17:04:59,478] Trial 1 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 181, 'max_depth': 5}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-08-17 17:05:00,376] Trial 2 finished with value: 0.7616387337057727 and parameters: {'n_estimators': 112, 'max_depth': 3}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-08-17 17:05:00,988] Trial 3 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 80, 'max_depth': 19}. Best is trial 0 with value: 0.7728119180633147.
[I 2025-08-17 17:05:01,767] Trial 4 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 138, 'max_depth': 12}. Best is trial 0 with value: 0.7728119

In [12]:
# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7802607076350093
Best hyperparameters: {'n_estimators': 129, 'max_depth': 16}


In [14]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


In [15]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [16]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2025-08-17 17:11:28,831] A new study created in memory with name: no-name-2a1a7d6f-56f1-4b1f-80d7-5eb743e7f133
[I 2025-08-17 17:11:30,120] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-08-17 17:11:31,338] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2025-08-17 17:11:31,645] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-08-17 17:11:32,236] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2025-08-17 17:11:32,822] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [17]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [18]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')

Test Accuracy with best hyperparameters: 0.74


In [19]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [20]:
# 1. Optimization History
plot_optimization_history(study).show()

In [21]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [22]:
# 3. Slice Plot
plot_slice(study).show()

In [23]:
# 4. Contour Plot
plot_contour(study).show()

In [25]:
#  5. Hyperparameter Importance
plot_param_importances(study).show()

In [26]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [27]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [28]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2025-08-17 17:21:55,980] A new study created in memory with name: no-name-206a0561-d7b2-4146-9dfd-1ecacd090fc6
[I 2025-08-17 17:21:56,026] Trial 0 finished with value: 0.7243947858472998 and parameters: {'classifier': 'SVM', 'C': 41.10712611168215, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.7243947858472998.
[I 2025-08-17 17:21:56,358] Trial 1 finished with value: 0.7746741154562384 and parameters: {'classifier': 'RandomForest', 'n_estimators': 56, 'max_depth': 5, 'min_samples_split': 2, 'min_samples_leaf': 5, 'bootstrap': False}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-08-17 17:21:56,446] Trial 2 finished with value: 0.7001862197392924 and parameters: {'classifier': 'SVM', 'C': 70.87814607918814, 'kernel': 'sigmoid', 'gamma': 'scale'}. Best is trial 1 with value: 0.7746741154562384.
[I 2025-08-17 17:21:58,974] Trial 3 finished with value: 0.750465549348231 and parameters: {'classifier': 'RandomForest', 'n_estimators': 164, 'max_depth': 3, 'min_

In [29]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.12967907400656783, 'kernel': 'linear', 'gamma': 'auto'}
Best trial accuracy: 0.7895716945996275


In [30]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.724395,2025-08-17 17:21:55.982536,2025-08-17 17:21:56.026492,0 days 00:00:00.043956,41.107126,NaN,SVM,auto,rbf,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.774674,2025-08-17 17:21:56.027571,2025-08-17 17:21:56.358631,0 days 00:00:00.331060,NaN,False,RandomForest,NaN,NaN,NaN,5.0,5.0,2.0,56.0,COMPLETE
2,2,0.700186,2025-08-17 17:21:56.361342,2025-08-17 17:21:56.446711,0 days 00:00:00.085369,70.878146,NaN,SVM,scale,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
3,3,0.750466,2025-08-17 17:21:56.447981,2025-08-17 17:21:58.974009,0 days 00:00:02.526028,NaN,True,RandomForest,NaN,NaN,NaN,3.0,8.0,3.0,164.0,COMPLETE
4,4,0.756052,2025-08-17 17:21:58.975362,2025-08-17 17:22:00.139064,0 days 00:00:01.163702,NaN,True,RandomForest,NaN,NaN,NaN,19.0,6.0,2.0,169.0,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.789572,2025-08-17 17:22:33.227106,2025-08-17 17:22:33.259293,0 days 00:00:00.032187,0.154345,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.785847,2025-08-17 17:22:33.260165,2025-08-17 17:22:33.293787,0 days 00:00:00.033622,0.246320,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
97,97,0.785847,2025-08-17 17:22:33.294679,2025-08-17 17:22:33.327195,0 days 00:00:00.032516,0.197533,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.785847,2025-08-17 17:22:33.328140,2025-08-17 17:22:33.415091,0 days 00:00:00.086951,7.128641,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [31]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,79
RandomForest,11
GradientBoosting,10


In [32]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()


,value
params_classifier,
GradientBoosting,0.747486
RandomForest,0.763332
SVM,0.773378


In [33]:
# 1. Optimization History
plot_optimization_history(study).show()